In [ ]:
Your storage is almost full (90%). … Soon you won't be able to upload new files to Drive and send or receive emails in Gmail.Learn more
import time
from tqdm import tqdm
import csv
import json
import numpy as np
import argparse
import pandas as pd
import requests
from bs4 import BeautifulSoup
import concurrent.futures
from csv import reader
import timeout_decorator
import datetime
from pandas.io.json import json_normalize
import nltk
import re
import datefinder
from tqdm import tqdm
from urllib.request import urlopen, Request


subscription_key = #input subscription key
assert subscription_key
cx = #input cx
assert cx

search_url = "https://www.googleapis.com/customsearch/v1?"
import requests

headers={"key": subscription_key, "cx": cx}

from IPython.display import HTML

def transform(url):
    if url.split('.')[-1]=='pdf':
        full_text.append(' ')
        return 
    else:
        try:
            r = requests.get(str(url),timeout=2)
            full_text.append(extract_content(r))
            return
        except:
            full_text.append(' ')
            return

def extract_content(req):
    snippet = []
    for string in BeautifulSoup(req.text, "html.parser").stripped_strings:
        snippet.append(string)
    
    if len(snippet)==0:
        return ' '
    else:
        return ' '.join(snippet)

def date_snippet(snippet):
    #dictionary of months
    num_months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    alp_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_dict = dict(zip(num_months,alp_months))
    #print(month_dict)

    # a generator will be returned by the datefinder module. I'm typecasting it to a list. Please read the note of caution provided at the bottom.
    matches = list(datefinder.find_dates(snippet))

    if len(matches) > 0:
        # date returned will be a datetime.datetime object. here we are only using the first match.
        date = matches[0]
        year = date.strftime("%Y")
        month = date.strftime("%m")
        day = date.strftime("%d")
        if day[0] == '0':
            day = day.replace("0","",1)
        remove_date = str(month_dict[str(month)] + ' ' + day + ',' + ' ' + year)
        #print(remove_date in test[2])
        return remove_date

    else:
        return ""

def clean_data(snippet):
    exception = ". . .:(),'" #do not remove ...
    cleaned_test = re.sub(r'[^\w' + exception + ']', ' ', snippet)
    #print(cleaned_test)
    date_to_remove = date_snippet(snippet)
    if date_to_remove in cleaned_test:
        cleaned_test = cleaned_test.replace(date_to_remove, "")
        #print("pass")
        
    cleaned_test = cleaned_test.replace("...","",1)
    cleaned_test = cleaned_test.lstrip()
    if cleaned_test[0] == ".":
        cleaned_test = cleaned_test.replace("...","",1)
    cleaned_test = cleaned_test.strip()
    cleaned_test = " ".join(re.split("\s+", cleaned_test, flags=re.UNICODE))
    " ".join(cleaned_test.split())
    return cleaned_test

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


def grab_web_text(search_query, max_sentences):
    params = {"key": subscription_key, "cx": cx, "q": search_query.lower(), "num": max_sentences}
    response = requests.get(search_url, params=params)
    response.raise_for_status()
    search_results = response.json()
    urls = []
    snippets = []
    titles = []
    if "items" not in search_results:
        snippets = [search_query] * max_sentences
        return urls, snippets
    for v in search_results["items"]:
        if not isinstance(v['link'], str):
            v['link'] = v['link'].decode('utf-8')
        if not isinstance(v['title'], str):
            v['title'] = v['title'].decode('utf-8')

        urls.append(v['link'])
        titles.append(v['title'])

        if not isinstance(v["snippet"],str):
            v["snippet"] = v["snippet"].encode("utf-8")
        #if "datePublished" in v:
        #    print (v["datePublished"])

        snippets.append(v["snippet"].replace("<b>","").replace("</b>","").replace("\n",""))
        
    
    return urls, snippets, titles


"""
def live_crawl(id, query, limit):
    print ('Querying: {} ...'.format(query))

    js = {}
    urls, snippets, titles = grab_web_text(query, limit)

    js['id'] = id
    js['claim'] = query.lower()
    js['snippets'] = snippets
    js['titles'] = titles
    js['urls'] = urls

    return js
"""

if __name__ == '__main__':
    start = time.time()
    parser = argparse.ArgumentParser()
    parser.add_argument('--query_path', type=str, default='input.txt')
    parser.add_argument('--output_path', type=str, default='output.json')
    parser.add_argument('--limit', type=int, default=10)
    args = parser.parse_args()
    
    with open(args.query_path, 'r') as f:
        query = f.read().splitlines()
    
    count_query=0
    all_records = []
    for q in query:
        print ('Querying: {} ...'.format(q))
        urls, snippets, titles = grab_web_text(q, args.limit)
        
        complete_snippets = []
        full_text = []
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
          executor.map(transform, urls)
        
        
        count = 0
        
        while count < len(urls):
            extracted_snippet = []
            url = urls[count]
            incomp_snippet = snippets[count]
            full_content = full_text[count]

            cleaned_snippet = clean_data(incomp_snippet).lower().split('...')
            cleaned_snippet = list(filter(lambda string : string.strip(), cleaned_snippet))
            sentences_to_complete = len(cleaned_snippet)
            
            clean_text_sentences = split_into_sentences(full_content)
            
            sentences_completed = 0

            for sent in clean_text_sentences:

                if sentences_completed == sentences_to_complete:
                    break

                if len(sent) < len(cleaned_snippet[sentences_completed]):
                    continue

                actual_sentence = sent
                test_sentence = ''.join(filter(str.isalnum, sent.lower()))
                cleaned_snippet_sent = ''.join(filter(str.isalnum, cleaned_snippet[sentences_completed]))

                if cleaned_snippet_sent in test_sentence:
                    extracted_snippet.append(actual_sentence)
                    sentences_completed += 1
                    
            if len(extracted_snippet)==0:
                complete_snippets.append(' '.join(cleaned_snippet))
            
            else:
                print("#####Successfully retrieved!###########")
                completed_snippet = ' '.join(extracted_snippet)
                print("Complete snippet: ", completed_snippet)
                print()
                print('-'*100)
                complete_snippets.append(completed_snippet)
            
            count+=1
            
            
              
        js = {}
        js['id'] = count
        js['claim'] = q.lower()
        js['snippets'] = complete_snippets
        js['titles'] = titles
        js['urls'] = urls
        all_records.append(js)
          
        count_query += 1

    records = {'records': all_records}
    with open(args.output_path, 'w+') as g:
        g.write(json.dumps(records))
    
    end = time.time()
    print("Total time taken: ", end-start)
    